In [2]:
# Import neccesary libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
from typing import Dict, Text

In [4]:
ratings = tfds.load("movielens/100k-ratings", split = "train")
movie = tfds.load("movielens/100k-movies")

2022-08-15 13:01:08.818850: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/movielens/100k-ratings/0.1.1.incomplete32M5XI/movielens-train.tfrecord*...:   …

Dataset movielens downloaded and prepared to ~/tensorflow_datasets/movielens/100k-ratings/0.1.1. Subsequent calls will reuse this data.


2022-08-15 13:01:49.041933: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
ratings = ratings.map(lambda x: {
    "movie_title" : x["movie_title"],
    "user_id" : x["user_id"],
    "user_rating" : x["user_rating"] # We are going to keep the ratings, they are considered explicit feedback
})

In [7]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100000, seed = 42, reshuffle_each_iteration=False)

train = shuffled.take(80000)
test = shuffled.skip(80000).take(20000)